In [7]:
import pandas as pd

weather_data = pd.read_csv("../Data/combined_weatherdata_2022.csv", header = 0, sep=',')
#print(weather_data.head())

cutoff_rain_day = 0.0002

weather_data["bool_rainday"] = weather_data["LC_DAILYRAIN"] > cutoff_rain_day

print(weather_data.head())

data_noise = pd.read_csv('../Data/combined_noisedata_2022.csv', header=0, sep=',', parse_dates=["result_date"])
data_noise.head()

   Year  Month  Day  LC_HUMIDITY  LC_DWPTEMP       LC_n     LC_RAD  LC_RAININ   
0  2022      1    1    86.557205   10.257703  36.346609  21.264531   0.000002  \
1  2022      1    2    83.512460    9.379296  36.056435  14.140794   0.000658   
2  2022      1    3    89.579036    8.004425  35.882086   5.529369   0.000750   
3  2022      1    4    94.288450    5.983424  35.816041   7.944379   0.000469   
4  2022      1    5    88.026102    2.078484  35.573298  13.893690   0.000131   

   LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED      Hour     Minute   LC_RAD60   
0      0.000029   -6.567639      0.322477  11.58042  25.174825  21.396634  \
1      0.003468  -28.866315      0.721487  11.50000  25.000000  14.099483   
2      0.006831  -35.256016      0.663436  11.50000  25.000000   5.517430   
3      0.003571  -25.637291      0.333304  11.50000  25.000000   7.962571   
4      0.000262  -44.342911      0.609569  11.50000  25.000000  13.890103   

   LC_TEMP_QCL0  LC_TEMP_QCL1  LC_TEMP_QCL2  LC_TE

,index,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,0,2022-01-01,MP 03: Naamsestraat 62 Taste,51.665242,49.992637,59.345285,71.190601,1.0,1.0
1,1,2022-01-01,MP 05: Calvariekapel KU Leuven,48.747476,46.504067,54.970850,67.557919,1.0,1.0
2,2,2022-01-01,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220,54.286131,67.321329,1.0,1.0
3,3,2022-01-01,MP 07: Naamsestraat 81,45.908501,44.373056,55.526650,66.988945,1.0,1.0
4,4,2022-01-02,MP 03: Naamsestraat 62 Taste,51.407297,50.094018,59.816366,71.545452,1.0,2.0


In [4]:
import plotly.graph_objects as go

average_noise = data_noise.groupby('result_month')['laeq'].mean()

months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

fig = go.Figure(data=go.Bar(x=months, y=average_noise))

# Set the color of bars based on the value
#fig.update_traces(marker=dict(color=['red' if val < 0 else 'green' for val in average_noise]))

fig.update_layout(
    title="Average noise level per month (standardized data)",
    xaxis_title="Month",
    yaxis_title="Average noise level (Laeq)"
)

fig.show()

In [19]:
data_noise_merged = pd.merge(data_noise, weather_data[["Month", "Day", "bool_rainday"]], left_on=["result_month", "result_day"], right_on=["Month", "Day"], how="left")
print(data_noise_merged.head(10))

print(len(data_noise_merged))

print(len(data_noise))

data_noise_merged = data_noise_merged.drop_duplicates()

print(len(data_noise_merged))

   index result_date                       description      lamax       laeq   
0      0  2022-01-01      MP 03: Naamsestraat 62 Taste  51.665242  49.992637  \
1      0  2022-01-01      MP 03: Naamsestraat 62 Taste  51.665242  49.992637   
2      1  2022-01-01    MP 05: Calvariekapel KU Leuven  48.747476  46.504067   
3      1  2022-01-01    MP 05: Calvariekapel KU Leuven  48.747476  46.504067   
4      2  2022-01-01  MP 06: Parkstraat 2 La Filosovia  48.270005  46.007220   
5      2  2022-01-01  MP 06: Parkstraat 2 La Filosovia  48.270005  46.007220   
6      3  2022-01-01            MP 07: Naamsestraat 81  45.908501  44.373056   
7      3  2022-01-01            MP 07: Naamsestraat 81  45.908501  44.373056   
8      4  2022-01-02      MP 03: Naamsestraat 62 Taste  51.407297  50.094018   
9      5  2022-01-02    MP 05: Calvariekapel KU Leuven  47.959057  46.297931   

        lceq     lcpeak  result_month  result_day  Month  Day  bool_rainday  
0  59.345285  71.190601           1.0    

In [30]:
import plotly.graph_objects as go

rainy_data = data_noise_merged[data_noise_merged["bool_rainday"]]
dry_data = data_noise_merged[~data_noise_merged["bool_rainday"]]

print(rainy_data.head())

mean_rainy = rainy_data["laeq"].mean()
print(mean_rainy)

mean_total_data = data_noise["laeq"].mean()
print(mean_total_data)

mean_dry = dry_data["laeq"].mean()
print(mean_dry)

average_noise_rainy = rainy_data.groupby('result_month')['laeq'].mean()

months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

fig = go.Figure(data=go.Bar(x=months, y=average_noise_rainy))

# Set the color of bars based on the value
#fig.update_traces(marker=dict(color=['red' if val < 0 else 'green' for val in average_noise]))

fig.update_layout(
    title="Average noise level per month when it is raining",
    xaxis_title="Month",
    yaxis_title="Average noise level (Laeq)"
)

fig.show()

    index result_date                       description      lamax       laeq   
8       4  2022-01-02      MP 03: Naamsestraat 62 Taste  51.407297  50.094018  \
9       5  2022-01-02    MP 05: Calvariekapel KU Leuven  47.959057  46.297931   
10      6  2022-01-02  MP 06: Parkstraat 2 La Filosovia  46.794855  45.085579   
11      7  2022-01-02            MP 07: Naamsestraat 81  45.846088  44.653385   
12      8  2022-01-03      MP 03: Naamsestraat 62 Taste  53.213369  51.830320   

         lceq     lcpeak  result_month  result_day  Month  Day  bool_rainday  
8   59.816366  71.545452           1.0         2.0      1    2          True  
9   56.015343  68.140432           1.0         2.0      1    2          True  
10  54.340200  67.139457           1.0         2.0      1    2          True  
11  56.346897  67.807183           1.0         2.0      1    2          True  
12  61.097236  72.989467           1.0         3.0      1    3          True  
51.02336572934685
50.63506163476682
50.

In [32]:
average_noise_dry = dry_data.groupby('result_month')['laeq'].mean()

fig = go.Figure(data=go.Bar(x=months, y=average_noise_dry))

# Set the color of bars based on the value
#fig.update_traces(marker=dict(color=['red' if val < 0 else 'green' for val in average_noise]))

fig.update_layout(
    title="Average noise level per month when it is dry",
    xaxis_title="Month",
    yaxis_title="Average noise level (Laeq)"
)

fig.show()